# Adaptive Rounding (AdaRound)
This notebook shows a working code example of how to use AIMET to perform Adaptive Rounding (AdaRound).

AIMET quantization features typically use the "nearest rounding" technique for achieving quantization.
When using the "nearest rounding" technique, the weight value is quantized to the nearest integer value.

AdaRound optimizes a loss function using unlabeled training data to decide whether to quantize a specific weight to the closer integer value or the farther one.
Using AdaRound quantization, a model is able to achieve an accuracy closer to the FP32 model, while using low bit-width integer quantization.

#### Overall flow
This notebook covers the following:
1. Instantiate the example evaluation and training pipeline
2. Load the FP32 model and evaluate the model to find the baseline FP32 accuracy
3. Create a quantization simulation model (with fake quantization ops inserted) and evaluate this simuation model to get a quantized accuracy score
4. Apply AdaRound and evaluate the simulation model to get a post-finetuned quantized accuracy score

#### What this notebook is not
* This notebook is not designed to show state-of-the-art results
* For example, it uses a relatively quantization-friendly model like Resnet50
* Also, some optimization parameters are deliberately chosen to have the notebook execute more quickly

---
## Dataset

This notebook relies on the ImageNet dataset for the task of image classification.
If you already have a version of the dataset readily available, use that.
Otherwise, download the dataset from an appropriate location (e.g. https://image-net.org/challenges/LSVRC/2012/index.php#) and convert them into tfrecords.

**Note1**: The dataloader provided in this example notebook relies on the ImageNet tfrecords dataset having the following characteristics:
- A folder containing tfrecords files starting with **'train\*'** for training files and **'valid\*'** for validation files.
- Each tfrecord file should have features: **'image/encoded'** for image data and **'image/class/label'** for its corresponding class.

**Note2**: To speed up the execution of this notebook, you may use a reduced subset of the ImageNet dataset.
E.g. the entire ILSVRC2012 dataset has 1000 classes, 1000 training samples per class and 50 validation samples per class.
For the purpose of running this notebook, you could reduce the dataset to 2 samples per class and then convert it into tfrecords.
This exercise is left up to the reader and is not necessary.

Edit the cell below and specify the directory where the downloaded ImageNet dataset is saved.

In [ ]:
TFRECORDS_DIR = '/path/to/dataset/'        # Please replace this with a real directory

We disable logs at the INFO level and disable eager execution.
We set verbosity to the level as displayed (ERROR), so TensorFlow will display all messages that have the label ERROR (or more critical).

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.logging.set_verbosity(tf.logging.ERROR)

---
## 1. Example Evaluation and Training Pipeline

The following is an example training and validation loop for this image classification task.

- **Does AIMET have any limitations on how the training, validation pipeline is written?**

    Not really. We will see later that AIMET will modify the user's model to create a QuantizationSim model which is still a TensorFlow model.
    This QuantizationSim model can be used in place of the original model when doing inference or training.

- **Does AIMET put any limitation on the interface of the evaluate() or train() methods?**

    Not really. You should be able to use your existing evaluate and train routines as-is.


In [ ]:
from typing import List

from Examples.common import image_net_config
from Examples.tensorflow.utils.image_net_evaluator import ImageNetDataLoader
from Examples.tensorflow.utils.image_net_evaluator import ImageNetEvaluator
from Examples.tensorflow.utils.image_net_trainer import ImageNetTrainer

class ImageNetDataPipeline:
    """
    Provides APIs for model evaluation and finetuning using ImageNet Dataset.
    """

    @staticmethod
    def get_val_dataloader():
        """
        Instantiates a validation dataloader for ImageNet dataset and returns it
        """
        data_loader = ImageNetDataLoader(TFRECORDS_DIR,
                                         image_size=image_net_config.dataset['image_size'],
                                         batch_size=image_net_config.evaluation['batch_size'],
                                         format_bgr=True)

        return data_loader

    @staticmethod
    def evaluate(sess: tf.Session) -> float:
        """
        Given a TF session, evaluates its Top-1 accuracy on the validation dataset
        :param sess: The sess graph to be evaluated.
        :return: The accuracy for the sample with the maximum accuracy.
        """
        evaluator = ImageNetEvaluator(TFRECORDS_DIR, training_inputs=['keras_learning_phase:0'],
                                      data_inputs=['input_1:0'], validation_inputs=['labels:0'],
                                      image_size=image_net_config.dataset['image_size'],
                                      batch_size=image_net_config.evaluation['batch_size'],
                                      format_bgr=True)

        return evaluator.evaluate(sess)


    @staticmethod
    def finetune(sess: tf.Session, update_ops_name: List[str], epochs: int, learning_rate: float, decay_steps: int):
        """
        Given a TF session, finetunes it to improve its accuracy
        :param sess: The sess graph to fine-tune.
        :param update_ops_name: list of name of update ops (mostly BatchNorms' moving averages).
                                tf.GraphKeys.UPDATE_OPS collections is always used
                                in addition to this list
        :param epochs: The number of epochs used during the finetuning step.
        :param learning_rate: The learning rate used during the finetuning step.
        :param decay_steps: A number used to adjust(decay) the learning rate after every decay_steps epochs in training.
        """
        trainer = ImageNetTrainer(TFRECORDS_DIR, training_inputs=['keras_learning_phase:0'],
                                  data_inputs=['input_1:0'], validation_inputs=['labels:0'],
                                  image_size=image_net_config.dataset['image_size'],
                                  batch_size=image_net_config.train['batch_size'],
                                  num_epochs=epochs, format_bgr=True)

        trainer.train(sess, update_ops_name=update_ops_name, learning_rate=learning_rate, decay_steps=decay_steps)


---
## 2. Load the model and evaluate to get a baseline FP32 accuracy score

For this example notebook, we are going to load a pretrained ResNet50 model from keras and covert it to a tensorflow session.
Similarly, you can load any pretrained tensorflow model instead.

Calling clear_session() releases the global state: this helps avoid clutter from old models and layers, especially when memory is limited.

By default the update ops are placed in tf.GraphKeys.UPDATE_OPS, so they need to be added as a dependency to the train_op.
Since batchnorm ops are folded, these need to be ignored during training.

In [ ]:
from tensorflow.compat.v1.keras.applications.resnet import ResNet50

tf.keras.backend.clear_session()

model = ResNet50(weights='imagenet', input_shape=(224, 224, 3))
update_ops_name = [op.name for op in model.updates] # Used for finetuning

The following utility method in AIMET sets BN layers in the model to eval mode.
This allows AIMET to more easily read the BN parameters from the graph.
Eventually we will fold BN layers into adjacent conv layers.

In [ ]:
from aimet_tensorflow.utils.graph import update_keras_bn_ops_trainable_flag

model = update_keras_bn_ops_trainable_flag(model, load_save_path="./", trainable=False)

AIMET features currently support tensorflow sessions.
**add_image_net_computational_nodes_in_graph** adds an output layer, softmax and loss functions to the Resnet50 model graph.

In [ ]:
from Examples.tensorflow.utils.add_computational_nodes_in_graph import add_image_net_computational_nodes_in_graph

sess = tf.keras.backend.get_session()

# Creates the computation graph of ResNet within the tensorflow session.
add_image_net_computational_nodes_in_graph(sess, model.output.name, image_net_config.dataset['images_classes'])

Since all tensorflow input and output tensors have names, we identify the tensors needed by AIMET APIs here.

In [ ]:
starting_op_names = [model.input.name.split(":")[0]]
output_op_names = [model.output.name.split(":")[0]]

We are checking if TensorFlow is using CPU or CUDA device.
This example code will use CUDA if available in your current execution environment.

In [ ]:
use_cuda = tf.test.is_gpu_available(cuda_only=True)

---
Let's determine the FP32 (floating point 32-bit) accuracy of this model using the evaluate() routine:

In [ ]:
accuracy = ImageNetDataPipeline.evaluate(sess=sess)
print(accuracy)

---
## 3. Create a quantization simulation model and determine quantized accuracy

## Fold Batch Normalization layers
Before we determine the simulated quantized accuracy using QuantizationSimModel, we will fold the BatchNormalization (BN) layers in the model.
These layers get folded into adjacent Convolutional layers. The BN layers that cannot be folded are left as they are.

**Why do we need to this?**

On quantized runtimes (like TFLite, SnapDragon Neural Processing SDK, etc.), it is a common practice to fold the BN layers.
Doing so results in an inferences/sec speedup since unnecessary computation is avoided.

From a floating point compute perspective, a BN-folded model is mathematically equivalent to a model with BN layers from an inference perspective, and produces the same accuracy.
However, folding the BN layers can increase the range of the tensor values for the weight parameters of the adjacent layers.

This can have a negative impact on the quantized accuracy of the model (especially when using INT8 or lower precision).
We want to simulate that on-target behavior by doing BN folding here.

The following code calls AIMET to fold the BN layers on the given model and returns a new session:

In [ ]:
from aimet_tensorflow.batch_norm_fold import fold_all_batch_norms

BN_folded_sess, _= fold_all_batch_norms(sess,
                                        input_op_names=starting_op_names,
                                        output_op_names=output_op_names)

## Create Quantization Sim Model

Now we use AIMET to create a QuantizationSimModel.

Before we create the QuantizationSimModel, we save and load a version of the BN folded session for QuantSim to use.
QuantSim will insert fake quantization ops in the session passed into it, and we want to maintain a fresh copy of the BN folded session for use in AdaRound later.

In [ ]:
from aimet_tensorflow.utils.graph_saver import save_and_load_graph
BN_folded_sess_copy = save_and_load_graph("output", BN_folded_sess)

AIMET will insert fake quantization ops in the model graph and configure them.
A few of the parameters are explained here:
- **quant_scheme**:
    - We set this to "post_training_tf_enhanced"
      With this choice of quant scheme, AIMET will use the TF Enhanced quant scheme to initialize the quantization parameters like scale/offset.
- **default_output_bw**: Setting this to 8 means that we are asking AIMET to perform all activation quantizations in the model using integer 8-bit precision.
- **default_param_bw**: Setting this to 8 means that we are asking AIMET to perform all parameter quantizations in the model using integer 8-bit precision.

There are other parameters that are set to default values in this example.
Please check the AIMET API documentation of QuantizationSimModel to see reference documentation for all the parameters.

In [ ]:
from aimet_common.defs import QuantScheme
from aimet_tensorflow.quantsim import QuantizationSimModel

sim = QuantizationSimModel(session=BN_folded_sess_copy,
                           starting_op_names=starting_op_names,
                           output_op_names=output_op_names,
                           quant_scheme= QuantScheme.post_training_tf_enhanced,
                           default_output_bw=8,
                           default_param_bw=8,
                           use_cuda=use_cuda)

---
## Compute Encodings
Even though AIMET has added 'quantizer' nodes to the model graph, the model is not ready to be used yet.
Before we can use the sim model for inference or training, we need to find appropriate scale/offset quantization parameters for each 'quantizer' node.

For activation quantization nodes, we need to pass unlabeled data samples through the model to collect range statistics which will then let AIMET calculate appropriate scale/offset quantization parameters.
This process is sometimes referred to as calibration. AIMET simply refers to it as 'computing encodings'.

We create a routine to pass unlabeled data samples through the model.
This should be fairly simple - use the existing train or validation data loader to extract some samples and pass them to the model.
We don't need to compute any loss metrics, so we can just ignore the model output. A few pointers regarding the data samples:

- In practice, we need a very small percentage of the overall data samples for computing encodings.
  For example, the training dataset for ImageNet has 1M samples. For computing encodings we only need 500 to 1000 samples.
- It may be beneficial if the samples used for computing encoding are well distributed.
  It's not necessary that all classes need to be covered since we are only looking at the range of values at every layer activation.
  However, we definitely want to avoid an extreme scenario like all 'dark' or 'light' samples are used - e.g. only using pictures captured at night might not give ideal results.

The following shows an example of a routine that passes unlabeled samples through the model for computing encodings.
This routine can be written in many different ways, this is just an example.

In [ ]:
def pass_calibration_data(session: tf.Session, _):
    data_loader = ImageNetDataPipeline.get_val_dataloader()
    batch_size = data_loader.batch_size

    input_label_tensors = [session.graph.get_tensor_by_name('input_1:0'),
                           session.graph.get_tensor_by_name('labels:0')]

    train_tensors = [session.graph.get_tensor_by_name('keras_learning_phase:0')]
    train_tensors_dict = dict.fromkeys(train_tensors, False)

    eval_outputs = [session.graph.get_operation_by_name('top1-acc').outputs[0]]

    samples = 500

    batch_cntr = 0
    for input_label in data_loader:
        input_label_tensors_dict = dict(zip(input_label_tensors, input_label))

        feed_dict = {**input_label_tensors_dict, **train_tensors_dict}

        with session.graph.as_default():
            _ = session.run(eval_outputs, feed_dict=feed_dict)

        batch_cntr += 1
        if (batch_cntr * batch_size) > samples:
            break


---
Now we call AIMET to use the above routine to pass data through the model and then subsequently compute the quantization encodings.
Encodings here refer to scale/offset quantization parameters.

In [ ]:
sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=None)

---
Now the QuantizationSim model is ready to be used for inference or training.
We can pass this model to the same evaluation routine we used before.
The evaluation routine will now give us a simulated quantized accuracy score for INT8 quantization instead of the FP32 accuracy score we saw before.

In [ ]:
accuracy = ImageNetDataPipeline.evaluate(sim.session)
print(accuracy)

---
## 4. Apply AdaRound

We can now apply AdaRound to this model.

Some of the parameters for AdaRound are described below:

- **data_loader:**  AdaRound needs a dataloader to use data samples for the layer-by-layer optimization to learn the rounding vectors.
  Either a training or validation dataloader could be passed in.
- **num_batches:** The number of batches used to evaluate the model while calculating the quantization encodings.
  Typically we want AdaRound to use around 2000 samples. So with a batch size of 32, this translates to 64 batches.
  To speed up the execution here we are using 5 batches.
- **default_num_iterations:** The number of iterations to AdaRound each layer.
  Default value is set to 10000 and we strongly recommend to not reduce this number.
  But in this example we are using 32 to speed up the execution runtime.
- **path:** The path where AdaRound parameter encodings are exported. Ensure that this folder exists prior to calling the API.

In [ ]:
from aimet_tensorflow.adaround.adaround_weight import Adaround
from aimet_tensorflow.adaround.adaround_weight import AdaroundParameters

num_batches = 5
num_iterations = 32
data_set = ImageNetDataPipeline.get_val_dataloader().dataset
params = AdaroundParameters(data_set=data_set, num_batches=num_batches, default_num_iterations=num_iterations)
ada_model = Adaround.apply_adaround(BN_folded_sess, starting_op_names=starting_op_names,
                                    output_op_names=output_op_names, params=params,
                                    path="output", filename_prefix="adaround", default_param_bw=8,
                                    default_quant_scheme=QuantScheme.post_training_tf_enhanced)

---
Now, we can determine the simulated quantized accuracy of the model after applying AdaRound.
We again create a simulation model like before and evaluate to determine simulated quantized accuracy.

**Note:** There are two important things to understand in the following cell.
  - **Parameter Biwidth Precision**: The QuantizationSimModel must be created with the same parameter bitwidth precision that was used in the apply_adaround() created.

  - **Freezing the parameter encodings**:
After creating the QuantizationSimModel, the set_and_freeze_param_encodings() API must be called before calling the compute_encodings() API.
While applying AdaRound, the parameter values have been rounded up or down based on these initial encodings internally created.
For Quantization Simulation accuracy, it is important to freeze these encodings.
If the parameters encodings are NOT frozen, the call to compute_encodings() will alter the value of the parameters encodings and Quantization Simulation accuracy will not reflect the AdaRounded accuracy.

In [ ]:
sim = QuantizationSimModel(session=ada_model,
                           starting_op_names=starting_op_names,
                           output_op_names=output_op_names,
                           quant_scheme= QuantScheme.post_training_tf_enhanced,
                           default_output_bw=8,
                           default_param_bw=8,
                           use_cuda=use_cuda)

sim.set_and_freeze_param_encodings(encoding_path=os.path.join("output", "adaround.encodings"))

sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=None)

---
Now the QuantizationSim model is ready to be used for inference.
First we can pass this model to the same evaluation routine we used before.
The evaluation routine will now give us a simulated quantized accuracy score for INT8 quantization, using the newly AdaRounded model with updated parameters.

In [ ]:
accuracy = ImageNetDataPipeline.evaluate(sim.session)
print(accuracy)

---
Depending on your settings you may have observed a slight gain in accuracy after applying AdaRound.
The settings used in this notebook are designed only to serve as code examples, designed to run quickly, but may not be optimal.
Please try this workflow against the model of your choice and play with the number of samples and other parameters to get the best results.

The next step would be to take this model to target.
We need to do two things:
- export the model with the updated weights without the fake quantization ops
- export the encodings (scale/offset quantization parameters).
AIMET QuantizationSimModel provides an export API for this purpose.

In [ ]:
sim.export(path="./output/", filename_prefix="resnet18_after_adaround")

---
## Summary

This example illustrated how the AIMET AdaRound API is invoked to achieve post training quantization.
To use AIMET AdaRound for your specific needs, replace the model with your model and replace the data pipeline with your data pipeline.
As indicated above, some parameters in this example have been chosen in such a way to make this example execute faster.

We hope this notebook was useful for you to understand how to use AIMET for performing AdaRound.

A few additional resources:
- Refer to the AIMET API docs to know more details of the APIs and optional parameters
- Refer to the other example notebooks to understand how to use AIMET post-training quantization techniques and QAT techniques